# Discussion Week 2

In this discussion we review vector and matrix operations. You will need to work some of these problems out by hand as well as to evaluate them in the Jupyter notebook. 

You can use the Shared Computing Cluster (SCC) or Google Colab to run this notebook.

The general instructions for running on the SCC are available under General Resources on [Piazza](https://piazza.com/bu/fall2025/ds722/resources).

## Inner Products

Given the vectors 

$$
\mathbf{x} = \begin{bmatrix} 2 \\ 2 \\ -1\end{bmatrix}, \quad \mathbf{y}= \begin{bmatrix} 2 \\ -1 \\ 2\end{bmatrix},
$$

compute by hand

1. $\mathbf{x}^{T}\mathbf{y}$
1. $\Vert \mathbf{x}\Vert_{2}$ and $\Vert\mathbf{y}\Vert_{2}$
1. the angle between $\mathbf{x}$ and $\mathbf{y}$

Using these values confirm that $\vert \mathbf{x}^{T}\mathbf{y}\vert\leq\Vert \mathbf{x}\Vert_{2}\Vert \mathbf{y}\Vert_{2}$.

Next verify your hand calculations in the following code cell. Create 1-D NumPy arrays for the vectors $\mathbf{x}$ and $\mathbf{y}$, then compute

1. $\mathbf{x}^{T}\mathbf{y}$ using  `np.dot(x,y)`
1. $\Vert \mathbf{x}\Vert_{2}$ and $\Vert\mathbf{y}\Vert_{2}$ using `np.linalg.norm(x)` and `np.linalg.norm(y)`
1. Use `np.acos` to compute the angle (`np.acos` is the inverse cosine function).

What can we conclude about these vectors?

In [ ]:
#TODO

## Matrix Vector Multiplication

Consider the matrix and vector

$$
A = 
\begin{bmatrix}
2 & -1 & 3 & 4 \\
1 & 2 & -3 & 2 \\
-1 & 1 & 2 & -2 \\
\end{bmatrix}\in\mathbb{R}^{3\times 4},\quad
\mathbf{x}
=
\begin{bmatrix}
1 \\
2\\
3\\
4\\
\end{bmatrix}\in\mathbb{R}^{4}.
$$

Compute by hand $b_{1}=A\mathbf{x}$.

In the following code cell, use NumPy to create the matrix and vector and verify your hand calculation using the `@` operator to perform the matrix-vector multiplication.

In addition, implement the matrix-vector multiplication as a sum of columns, i.e., code

$$
b_{2} = \sum_{j=1}^{4}\mathbf{a}_{j}x_{j},
$$

and verify $b_{1}=b_{2}$. You can get the $j$-th column of $A$ in numpy using `A[:,j]`.

In [ ]:
## TODO

## Matrix-Matrix Multiplication

Consider the matrices

$$
A = 
\begin{bmatrix}
2 & -1 & 3 & 4 \\
1 & 2 & -3 & 2 \\
-1 & 1 & 2 & -2 \\
\end{bmatrix}\in\mathbb{R}^{3\times 4},\quad
A^{T}
=
\begin{bmatrix}
2 & 1 & -1 \\
-1 & 2 & 1 \\
3 & -3 & 2 \\
4 & 2 & -2 \\
\end{bmatrix}\in\mathbb{R}^{4\times 3}.
$$

Compute $C=AA^{T}$ by hand. 

Then verify using NumPy that:

1. Each column $j$ of $C$ is $C[:, j] = \sum_{k=1}^{4}A[:, k]*B[k, j]$ for $j=1, 2, 3$
1. Each row $i$ of $C$ is $C[i, :] = \sum_{k=1}^{4}B[k, :]*A[i, k]$ for $i=1, 2, 3$
1. That $C$ can be expressed as a sum of outer products $C = \sum_{k=1}^{4} np.outer(A[:, k], B[k, :])$

What property does $C$ have?

In [ ]:
#TODO

## Rotations in 2D 

Recall from lecture that matrix-vector multiplication transforms one vector into another vector. This is an important operation, for instance, when rotating objects in a computer game. There is a specific 2D matrix for rotating vectors of length 2 (points in the x-y plane) counterclockwise by an angle $\theta$ in radians. The matrix has the form:

$$
G =
\begin{pmatrix}
\cos{\theta} & -\sin{\theta} \\
\sin{\theta} & \cos{\theta}
\end{pmatrix}
$$.

Computing the matrix vector product $Gv$ rotates the vector $v$ counterclockwise by the angle $\theta$.

In this problem you need to code several functions that will allow you to rotate a vector by a specified degree. You will then write code to visualize how this vector rotates in the plane.

1. Write a function `create_rotation_matrix()` that takes in an angle in degrees. The output should be the rotation matrix corresponding to this angle using the above formula. Remember that to convert from degrees to radians you multiple by a factor of $\pi/180$. You may use either math or NumPy to import the fucntions $\cos$, $\sin$, and constant $\pi$. Test your function using an angle of $45^{\circ}$ and print the returned matrix, which should be np.arrray([[ 0.70710678 -0.70710678],[ 0.70710678  0.70710678]]).

In [ ]:
# TODO

2. Using your `create_rotation_matrix` function, write a new function `rotate_vector(vector, angle)` that takes as input a 2D vector and an angle in degrees, and returns the rotated vector. (Do not call a rotation function to do the rotation; use your matrix.)  Test your function using the angle $45^{\circ}$ and the vector $[1, 0]$, printing the rotated vector.

In [ ]:
# TODO

3. We now have functions to create a rotation matrix based on an angle and a function to rotate a vector by that angle. Write a new function called `rotate_vectors(vector, angles)` that takes a vector and a list of angles in degrees. This function should return a list containing the original input vector rotated by each angle in the list of angles. Test this function using the vector `[1, 0]` and the list of angles `[0, 30, 60, 90, 120, 150, 180]`. Print the output list of vectors.

In [ ]:
# TODO

4. To visualize a vector [x, y] in the plane, we can use matplotlib to plot a line from the origin [0, 0] to the point [x, y]. Given the vector [-1, 1], plot a red line representing this vector.

In [ ]:
#TODO

5. We want to visualize the rotation of our initial vector. To do this write a helper function called `extract_coordinates(vectors)` which takes as input a list of 2D vectors. This function should return 2 numpy arrays that contain the x-coordinates and y-coordinates, respectively. Test this function using your vector of outputs from the previous cell. Print out the x-coordinates and the y-coordinates.

In [ ]:
# TODO

6. Let's put everything together now to visualize our rotations. You are given a starting vector `[1, 0]` and the following list of angles `[0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]`. Use your previous functions to get the x- and y-coordinates from `extract_coordinates`.  Then plot the rotated vectors in the different colors specified by the `colors` array.

In [ ]:
import matplotlib.pyplot as plt
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.grid()
colors = [
    "crimson", "teal", "gold", "indigo", "coral", 
    "limegreen", "orchid", "sienna", "dodgerblue", "plum",
    "turquoise", "magenta"
]

vector = np.array([1, 0])
angles = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]

# Use your previously created functions to get a list of x and y coordinates of the rotated vectors

vectors = # TODO
x_coords, y_coords = # TODO

for i, xy in enumerate(zip(x_coords, y_coords)):
    # Fill in the correct plot command to plot the xy coordinates of each
    # TODO
plt.show()